In [3]:
import os
from delta.tables import DeltaTable
from pyspark.sql import SparkSession
from dateutil.parser import parse
import pyspark.sql.types as T
import pyspark.sql.functions as F
import pandas as pd
import psycopg2
import requests

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
try:
    spark.stop()
except Exception as e: 
    print(e)

spark = (
    SparkSession.builder \
    .appName("SparkLocalStackS3Integration") \
    .config("spark.hadoop.fs.s3a.endpoint", os.environ['AWS_ENDPOINT_URL'])\
    .config("spark.hadoop.fs.s3a.access.key", os.environ["AWS_ACCESS_KEY_ID"])\
    .config("spark.hadoop.fs.s3a.secret.key",os.environ["AWS_SECRET_ACCESS_KEY"])\
    .config("spark.hadoop.fs.s3a.path.style.access", "true")\
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")\
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")\
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")\
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .getOrCreate()
)

In [5]:
pipeline_config_id = 1
DATALAKE_PATH = requests.get(f"http://backend:8000/configs/{pipeline_config_id}/").json()["upsert_write_path"]
print(DATALAKE_PATH)


s3a://my-storage-bucket/upsert-locations/postgres.public.task_instance_1_1


In [6]:
datalake_df = spark.read.format('delta').load(DATALAKE_PATH)

In [8]:
datalake_df.limit(10).toPandas()

,__op,custom_operator_name,dag_id,duration,end_date,executor,executor_config,external_executor_id,hostname,job_id,map_index,max_tries,next_kwargs,next_method,operator,pid,pool,pool_slots,priority_weight,queue,queued_by_job_id,queued_dttm,rendered_map_index,run_id,start_date,state,task_display_name,task_id,trigger_id,trigger_timeout,try_number,unixname,updated_at,__key,__topic,__partition,__offset,__timestamp,__timestampType,year,month,day
0,u,None,continuous_python_dag,0.174838,2025-06-15T12:59:36.407383Z,None,gAV9lC4=,91ee0479-693b-4f81-8f11-20e90b4e6531,02D4ADBE5286,19,-1,1,None,None,PythonOperator,126,default_pool,1,1,default,1,2025-06-15T12:59:35.999556Z,None,scheduled__2025-06-15T12:59:33.318357+00:00,2025-06-15 12:59:36.232545,success,run_my_continuous_task,run_my_continuous_task,None,None,1,root,2025-06-15T12:59:36.418720Z,"{""dag_id"":""continuous_python_dag"",""task_id"":""r...",postgres.public.task_instance,0,125,2025-06-15 12:59:37.011,0,2025,6,15
1,u,None,frequency_one,NaN,None,None,gAV9lC4=,f9656b80-14f9-4f0a-9c03-44e801a913e5,02D4ADBE5286,40,-1,0,None,None,DockerOperator,196,default_pool,1,2,default,1,2025-06-15T13:00:27.716155Z,None,manual__2025-06-15T13:00:26.638408+00:00,2025-06-15 13:00:28.214312,running,docker_task_append_1_postgres.public.task_inst...,docker_task_append_1_postgres.public.task_inst...,None,None,1,root,2025-06-15T13:00:28.315411Z,"{""dag_id"":""frequency_one"",""task_id"":""docker_ta...",postgres.public.task_instance,0,272,2025-06-15 13:00:28.745,0,2025,6,15
2,u,None,continuous_python_dag,0.104506,2025-06-15T13:00:08.818303Z,None,gAV9lC4=,811aaaec-5b03-449a-96b8-949b087f4b0a,02D4ADBE5286,35,-1,1,None,None,PythonOperator,181,default_pool,1,1,default,1,2025-06-15T13:00:08.560704Z,None,scheduled__2025-06-15T13:00:05.746418+00:00,2025-06-15 13:00:08.713797,success,run_my_continuous_task,run_my_continuous_task,None,None,1,root,2025-06-15T13:00:08.825288Z,"{""dag_id"":""continuous_python_dag"",""task_id"":""r...",postgres.public.task_instance,0,237,2025-06-15 13:00:09.165,0,2025,6,15
3,u,None,continuous_python_dag,0.148926,2025-06-15T12:59:12.461137Z,None,gAV9lC4=,5039d7cd-7cfb-4aa8-8db2-278e55a6ec34,02D4ADBE5286,6,-1,1,None,None,PythonOperator,80,default_pool,1,1,default,1,2025-06-15T12:59:11.909602Z,None,scheduled__2025-06-15T12:59:09.089809+00:00,2025-06-15 12:59:12.312211,success,run_my_continuous_task,run_my_continuous_task,None,None,1,root,2025-06-15T12:59:12.468230Z,"{""dag_id"":""continuous_python_dag"",""task_id"":""r...",postgres.public.task_instance,0,34,2025-06-15 12:59:12.869,0,2025,6,15
4,u,None,continuous_python_dag,0.104432,2025-06-15T13:00:04.920075Z,None,gAV9lC4=,c88afbb5-efaf-411f-9a68-495f6034411c,02D4ADBE5286,33,-1,1,None,None,PythonOperator,175,default_pool,1,1,default,1,2025-06-15T13:00:04.655591Z,None,scheduled__2025-06-15T13:00:01.375969+00:00,2025-06-15 13:00:04.815643,success,run_my_continuous_task,run_my_continuous_task,None,None,1,root,2025-06-15T13:00:04.927981Z,"{""dag_id"":""continuous_python_dag"",""task_id"":""r...",postgres.public.task_instance,0,223,2025-06-15 13:00:05.651,0,2025,6,15
5,u,None,continuous_python_dag,0.104709,2025-06-15T12:59:21.180500Z,None,gAV9lC4=,a791b0e1-f5ed-4e32-bbbf-1f34957aaf39,02D4ADBE5286,11,-1,1,None,None,PythonOperator,95,default_pool,1,1,default,1,2025-06-15T12:59:20.911597Z,None,scheduled__2025-06-15T12:59:18.134307+00:00,2025-06-15 12:59:21.075791,success,run_my_continuous_task,run_my_continuous_task,None,None,1,root,2025-06-15T12:59:21.187153Z,"{""dag_id"":""continuous_python_dag"",""task_id"":""r...",postgres.public.task_instance,0,69,2025-06-15 12:59:21.416,0,2025,6,15
6,u,None,continuous_python_dag,0.185700,2025-06-15T13:00:13.010174Z,None,gAV9lC4=,e20a2b23-1502-48e7-ab12-af20951ae2a1,02D4ADBE5286,37,-1,1,None,None,PythonOperator,187,default_pool,1,1,default,1,2025-06-15T13:00:12.585618Z,None,scheduled__2025-06-15T13:00:09.652077+00:00,2025-06-15 13:00:12.824474,success,run_my_continuous_task,run_my_continuous_task,None,None,1,r

In [10]:
spark.sql(f"""select * from delta.`{DATALAKE_PATH}` limit 10""").toPandas()

,__op,custom_operator_name,dag_id,duration,end_date,executor,executor_config,external_executor_id,hostname,job_id,map_index,max_tries,next_kwargs,next_method,operator,pid,pool,pool_slots,priority_weight,queue,queued_by_job_id,queued_dttm,rendered_map_index,run_id,start_date,state,task_display_name,task_id,trigger_id,trigger_timeout,try_number,unixname,updated_at,__key,__topic,__partition,__offset,__timestamp,__timestampType,year,month,day
0,u,None,continuous_python_dag,0.174838,2025-06-15T12:59:36.407383Z,None,gAV9lC4=,91ee0479-693b-4f81-8f11-20e90b4e6531,02D4ADBE5286,19,-1,1,None,None,PythonOperator,126,default_pool,1,1,default,1,2025-06-15T12:59:35.999556Z,None,scheduled__2025-06-15T12:59:33.318357+00:00,2025-06-15 12:59:36.232545,success,run_my_continuous_task,run_my_continuous_task,None,None,1,root,2025-06-15T12:59:36.418720Z,"{""dag_id"":""continuous_python_dag"",""task_id"":""r...",postgres.public.task_instance,0,125,2025-06-15 12:59:37.011,0,2025,6,15
1,u,None,frequency_one,NaN,None,None,gAV9lC4=,f9656b80-14f9-4f0a-9c03-44e801a913e5,02D4ADBE5286,40,-1,0,None,None,DockerOperator,196,default_pool,1,2,default,1,2025-06-15T13:00:27.716155Z,None,manual__2025-06-15T13:00:26.638408+00:00,2025-06-15 13:00:28.214312,running,docker_task_append_1_postgres.public.task_inst...,docker_task_append_1_postgres.public.task_inst...,None,None,1,root,2025-06-15T13:00:28.315411Z,"{""dag_id"":""frequency_one"",""task_id"":""docker_ta...",postgres.public.task_instance,0,272,2025-06-15 13:00:28.745,0,2025,6,15
2,u,None,continuous_python_dag,0.104506,2025-06-15T13:00:08.818303Z,None,gAV9lC4=,811aaaec-5b03-449a-96b8-949b087f4b0a,02D4ADBE5286,35,-1,1,None,None,PythonOperator,181,default_pool,1,1,default,1,2025-06-15T13:00:08.560704Z,None,scheduled__2025-06-15T13:00:05.746418+00:00,2025-06-15 13:00:08.713797,success,run_my_continuous_task,run_my_continuous_task,None,None,1,root,2025-06-15T13:00:08.825288Z,"{""dag_id"":""continuous_python_dag"",""task_id"":""r...",postgres.public.task_instance,0,237,2025-06-15 13:00:09.165,0,2025,6,15
3,u,None,continuous_python_dag,0.148926,2025-06-15T12:59:12.461137Z,None,gAV9lC4=,5039d7cd-7cfb-4aa8-8db2-278e55a6ec34,02D4ADBE5286,6,-1,1,None,None,PythonOperator,80,default_pool,1,1,default,1,2025-06-15T12:59:11.909602Z,None,scheduled__2025-06-15T12:59:09.089809+00:00,2025-06-15 12:59:12.312211,success,run_my_continuous_task,run_my_continuous_task,None,None,1,root,2025-06-15T12:59:12.468230Z,"{""dag_id"":""continuous_python_dag"",""task_id"":""r...",postgres.public.task_instance,0,34,2025-06-15 12:59:12.869,0,2025,6,15
4,u,None,continuous_python_dag,0.104432,2025-06-15T13:00:04.920075Z,None,gAV9lC4=,c88afbb5-efaf-411f-9a68-495f6034411c,02D4ADBE5286,33,-1,1,None,None,PythonOperator,175,default_pool,1,1,default,1,2025-06-15T13:00:04.655591Z,None,scheduled__2025-06-15T13:00:01.375969+00:00,2025-06-15 13:00:04.815643,success,run_my_continuous_task,run_my_continuous_task,None,None,1,root,2025-06-15T13:00:04.927981Z,"{""dag_id"":""continuous_python_dag"",""task_id"":""r...",postgres.public.task_instance,0,223,2025-06-15 13:00:05.651,0,2025,6,15
5,u,None,continuous_python_dag,0.104709,2025-06-15T12:59:21.180500Z,None,gAV9lC4=,a791b0e1-f5ed-4e32-bbbf-1f34957aaf39,02D4ADBE5286,11,-1,1,None,None,PythonOperator,95,default_pool,1,1,default,1,2025-06-15T12:59:20.911597Z,None,scheduled__2025-06-15T12:59:18.134307+00:00,2025-06-15 12:59:21.075791,success,run_my_continuous_task,run_my_continuous_task,None,None,1,root,2025-06-15T12:59:21.187153Z,"{""dag_id"":""continuous_python_dag"",""task_id"":""r...",postgres.public.task_instance,0,69,2025-06-15 12:59:21.416,0,2025,6,15
6,u,None,continuous_python_dag,0.185700,2025-06-15T13:00:13.010174Z,None,gAV9lC4=,e20a2b23-1502-48e7-ab12-af20951ae2a1,02D4ADBE5286,37,-1,1,None,None,PythonOperator,187,default_pool,1,1,default,1,2025-06-15T13:00:12.585618Z,None,scheduled__2025-06-15T13:00:09.652077+00:00,2025-06-15 13:00:12.824474,success,run_my_continuous_task,run_my_continuous_task,None,None,1,r